In [1]:
import shlex, subprocess, time
import runconfig

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
config_dict = runconfig.get_config_dict()
dataset_name = config_dict['dataset_name']
outlier_fraction = config_dict['outlier_fraction']
num_clients_per_class = config_dict['num_clients_per_class']
association_threshold = config_dict['association_threshold']

num_class = 10


l_ideal = []
for p1 in range(num_class):
    l2 = []
    for p2 in range(num_clients_per_class):  
        l2.append(str(p1)+"_"+str(p2))
    l_ideal.append(l2)
l_ideal
dataset_name

'fashion_mnist'

In [4]:
communities_file_name = dataset_name + str(num_clients_per_class) + '_communities.txt'
with open(communities_file_name) as file:
    lines = file.readlines()
    l_communities = [sorted(line.strip().replace(' ', '').split(',')) for line in lines]

for i in l_communities:
    print(i)

['0_0', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8']
['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8']
['3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8']
['2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '4_0', '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '6_0', '6_1', '6_2', '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']
['5_0', '5_1', '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8']
['7_0', '7_1', '7_2', '7_3', '7_4', '7_5', '7_6', '7_7', '7_8']
['8_0', '8_1', '8_2', '8_3', '8_4', '8_5', '8_6', '8_7', '8_8']
['9_0', '9_1', '9_2', '9_3', '9_4', '9_5', '9_6', '9_7']
['9_8']


In [5]:
%%capture
liss = l_communities; out_fold = 'communities' 
liss = l_ideal; out_fold = 'ideal' 

address = 5000 # is actually a port. Server and clients of the same group have the same port
for num,i in enumerate(liss[0:]):
    print(str(i))
    ll = ['./fed_start.sh', dataset_name, str(address+num), str(num_clients_per_class), out_fold] + i
    as_string = " ".join(ll)
    subprocess.check_call(as_string, shell=True)
    #subprocess.Popen(as_string, shell=True)
    #time.sleep(5)



Starting server


python: can't open file '/var/home/mirko/AutoFed3/src/fed_server.py': [Errno 2] No such file or directory


Starting client 0_0
Starting client 0_1
Starting client 0_2
Starting client 0_3
Starting client 0_4
Starting client 0_5
Starting client 0_6
Starting client 0_7
Starting client 0_8
END
Starting server


python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoFed3/src/fed_client.py': [Errno 2] No such file or directory
python: can't open file '/var/home/mirko/AutoF

: 

: 

In [ ]:
#dataset_name = 'mnist'
#dataset_name = 'fashion_mnist'
#num_clients_per_class = 18


pth = 'out'+str(num_clients_per_class)+'/'
#pth = 'out9/'

import pandas as pd
import glob
files = glob.glob(pth+"output_local/"+dataset_name+"/*.csv")

df1 = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f, index_col=[0])
    df1 = pd.concat([df1,csv])
    df1 = df1.sort_values('dataset')
df1 = df1.add_suffix('_local').reset_index(drop=True)

In [ ]:
import pandas as pd
import glob
files = glob.glob(pth+"output_communities/"+dataset_name+"/*.csv")

df2 = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f,index_col=[0])
    df2 = pd.concat([df2,csv])
    df2 = df2.sort_values('dataset')
df2 = df2.add_suffix('_clust').reset_index(drop=True)
df2

In [ ]:
import pandas as pd
import glob
files = glob.glob(pth+"output_ideal/"+dataset_name+"/*.csv")

df3 = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f,index_col=[0])
    df3 = pd.concat([df3,csv])
    df3 = df3.sort_values('dataset').reset_index(drop=True)
df3 = df3.add_suffix('_ideal').reset_index(drop=True)

In [ ]:
pd.concat([df1,df2,df3],axis=1)

In [ ]:
metric = 'f1in'
metric = 'f1out'
#metric = 'acc'
metric = 'aucroc'
pd.concat([df1,df2,df3],axis=1)
pd.concat([df1,df2,df3],axis=1)[[metric+'_local',metric+'_clust',metric+'_ideal']].reset_index(drop=True).style.highlight_max(color = 'lightgreen', axis = 1)
pd.concat([df1,df2,df3],axis=1)[[metric+'_local',metric+'_clust',metric+'_ideal']].reset_index(drop=True).mean()

In [ ]:
df2

In [ ]:
#df1['inlier'] = df2['dataset_ideal'].str.split('_', 1, expand=True)[0]
#df2['inlier'] = df2['dataset_ideal'].str.split('_', 1, expand=True)[0] # aggiungo la colonna inlier solo a 1
df3['inlier'] = df3['dataset_ideal'].str.split('_', 1, expand=True)[0]
pd.concat([df1,df2,df3],axis=1)[['inlier',metric+'_local',metric+'_clust',metric+'_ideal']].groupby('inlier').mean().style.highlight_max(color = 'lightgreen', axis = 1)